In [1]:
import pandas as pd
import useful_functions as use
from fuzzywuzzy import fuzz
import usaddress

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [41]:
bolos = pd.read_csv('../../Data/PhoneAppend/HSG_PHYS_BOLO_Address_2021-08-25.txt', sep='|')
bolos['ME'] = use.fix_me(bolos['me#'])

In [44]:
bolos = bolos.fillna('None')

In [46]:
bolos[bolos.addr_line_3!='None']

,entity_id,me#,comm_id,usage,load_type_ind,addr_type,addr_line_1,addr_line_3,addr_line_2,addr_city,addr_state,addr_zip,addr_plus4,addr_country,source,source_dtm,ME
0,10000103,4501810326,5711907,PO,R,OF,LEXINGTON CO HLTH SVS DIST INC,RADIOLOGY DEPARTMENT,627 SPRING LAKE RD,COLUMBIA,SC,29206,None,None,ASM,2021-08-25 13:26:27.136627,04501810326
2,10000324,4501810491,6225203,PO,R,OF,None,SOMERSET HLTH & WELLNESS C,2525 COURT DR,GASTONIA,NC,28054,None,None,ASM,2021-08-25 13:26:27.136627,04501810491
3,10000363,4501810521,11466799,PO,R,OF,None,STE 41,1130 PERRY HWY,PITTSBURGH,PA,15237,None,None,ASM,2021-08-25 13:26:27.136627,04501810521
4,10000415,4501810571,9098975,PO,R,OF,None,STE A,130 LAURENT WAY,IRMO,SC,29063,None,None,ASM,2021-08-25 13:26:27.136627,04501810571
5,10000441,4501810601,10504567,PO,R,OF,ST MARY'S HOSPITAL EMERGE,DEPARTMENT,PO BOX 692,WHITE PINE,TN,37890,None,None,ASM,2021-08-25 13:26:27.136627,04501810601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101601,9999011,4501801262,8540857,PO,R,OF,None,STE 101,9655 HOPEWELL RD,BOAZ,KY,42027,None,None,ASM,2021-08-25 13:26:27.136627,04501801262
101602,9999284,4501801475,6584398,PO,R,OF,None,GREENWOOD UROLOGICAL,126 COUNTRY CLUB DR,GREENWOOD,SC,29646,None,None,ASM,2021-08-25 13:26:27.136627,04501801475
101603,9999609,4501801769,6348109,PO,R,OF,"24ON PHYSICIANS, PC",THE REGIONAL MEDICAL CENTER,940 HOLLY ST,ORANGEBURG,SC,29115,None,None,ASM,2021-08-25 13:26:27.136627,04501801769
101605,9999804,4501810075,15817052,PO,R,OF,None,G. STEVEN AVIDON MD PA,PO BOX 621749,ORLANDO,FL,32862,None,None,ASM,2021-08-25 13:26:27.136627,04501810075


In [3]:
npi_me = use.get_npi_to_me()

In [4]:
new_download = '../../Data/DataGov/DAC_NationalDownloadableFile_21-08-12.csv'
new_file = pd.read_csv(new_download, encoding='latin', low_memory=False)
new_file.columns = [c.strip() for c in new_file.columns.values]
new_file['NPI'] = [str(int(x)) for x in new_file.NPI]

In [6]:
new_download_2 = '../../Data/DataGov/PhysicianComparePhoneNumberFile_21-08-12.csv'
phones = pd.read_csv(new_download_2, encoding='latin', low_memory=False)
phones.columns = [c.strip() for c in phones.columns.values]
phones['NPI'] = [str(int(x)) for x in phones.NPI]

In [7]:
phones = pd.merge(phones, npi_me, on='NPI')

In [8]:
gov = pd.merge(new_file, npi_me, on='NPI')

In [9]:
phoney = phones[['ME','adrs_id','phn_numbr','NPI']].drop_duplicates()

In [49]:
def is_a_match(thing_1, thing_2, threshold):
    if thing_1 == thing_2:
        match = True
    elif thing_1 in thing_2:
        match = True
    elif thing_2 in thing_1:
        match = True
    elif fuzz.ratio(thing_1, thing_2)>threshold:
        match = True
        print('')
        print(fuzz.ratio(thing_1, thing_2))
        print('------------------------------------')
    else:
        match = False
    return(match)
def fix(component):
    component = component.strip().upper()
    return component
def error_handle(parsed_string):
    new_dict = {}
    for thing in parsed_string:
        if thing[1] in new_dict.keys():
            a_list = [new_dict[thing[1]], thing[0]]
            new_dict[thing[1]] = max(a_list, key=len)
        else:
            new_dict[thing[1]] = thing[0]
    return(new_dict)
def clean_address_two(add_1):
    add_1 = fix(add_1)
    if add_1 == 'None':
        addr_1 = ''
    elif add_1 == 'NAN':
        addr_1 = ''
    else:
        addr_1 = ', ' + add_1
    return(addr_1)

def get_address_dict(address):
    try:
        address_dict = usaddress.tag(address)[0]
    except usaddress.RepeatedLabelError as e:
        print(e.original_string)
        address_dict = error_handle(e.parsed_string)
        print('')
    return address_dict
        
def parse_gov_address(moar):
    dict_list = []
    for row in moar.itertuples():
        addr_2 = clean_address_two(row.adr_ln_2)
        address = f'{fix(row.adr_ln_1)}{addr_2}, {fix(row.cty)}, {fix(row.st)}'
        address_dict = get_address_dict(address)
        address_dict['adrs_id'] = row.adrs_id
        address_dict['PHONE'] = row.phn_numbr
        address_dict['ZIPCODE'] = row.zip
        address_dict['ZIP'] = use.fix_zipcode(row.zip)
        dict_list.append(address_dict)
    parsed_df = pd.DataFrame(dict_list)
    parsed_df.dropna(how='all', axis=1, inplace=True)
    return (parsed_df)

['entity_id', 'me#', 'comm_id', 'usage', 'load_type_ind', 'addr_type',
       'addr_line_1', 'addr_line_3', 'addr_line_2', 'addr_city', 'addr_state',
       'addr_zip', 'addr_plus4', 'addr_country', 'source', 'source_dtm']

def parse_bolo_address(moar):
    dict_list = []
    for row in moar.itertuples():
        addr_2 = clean_address_two(row.addr_line_3)
        address = f'{fix(row.addr_line_2)}{addr_2}, {fix(row.addr_city)}, {fix(row.addr_state)}'
        address_dict = get_address_dict(address)
        address_dict['comm_id'] = row.comm_id
        address_dict['ZIPCODE'] = row.addr_zip
        address_dict['ZIP'] = use.fix_zipcode(row.addr_zip)
        dict_list.append(address_dict)
    parsed_df = pd.DataFrame(dict_list)
    parsed_df.dropna(how='all', axis=1, inplace=True)
    return (parsed_df)

In [11]:
gov = gov.fillna('None')
gov_adds = gov.drop_duplicates('adrs_id')
gov_parsed = parse_gov_address(gov_adds)

SUITE 408 TORRE SAN FRANCISCO, 369 DE DIEGO AVE, RIO PIEDRAS, PR

7625 WISCONSIN AVE, 250 APEX MEDICAL SERVICES, BETHESDA, MD

1260 SILAS DEANE HWY 110, NONE, WETHERSFIELD, CT

333 E CITY LINE AVE, 2 BALA PLAZA SUITE PL50, BALA CYNWYD, PA

1110 EASTERN PKWY, PIERRE TOUSSAINT FAM HLTH CTR, BROOKLYN, NY

CARR 185 KM 15, INSTITUTO DE MEDICINA DE FAMILIA DE CEDROS, CAROLINA, PR

CARR 2 KM, 4 HOSPITAL DE LA CONCEPCION, SAN GERMAN, PR

1475 WILSON AVE, WILSON MED BLDG OFIC 3B, SAN JUAN, PR

ROAD 21 URB LAS LOMAS, TORRE DEL METROPOLITANO 178 OFI 205 HOSPITAL METRO, SAN JUAN, PR

255 W LANCASTER AVE, MOB II SUITE 121, PAOLI, PA

5501 OLD YORK RD WCB 4TH FLR, EINSTEIN CT SURGERY ASSOC, PHILADELPHIA, PA

CARR 111 KM 2, BO PALMAR, AGUADILLA, PR

CARR 2 KM, 1 BO CANDELARIA, ARECIBO, PR

51 MENDEZ VIGO ST, CORNER OF PERAL ST, MAYAGUEZ, PR

3441 24TH AVE NW, 105 R BRUCE PARKER MD, NORMAN, OK

2323 S 171ST ST, 100 JOHN W PFLUG MD PC, OMAHA, NE

4911 W ST JOE HWY 102, NONE, LANSING, MI

1 SEARS DR, SU

AVE SEVERIANO CUEVAS, 18 CARR 460 CAIMITAL BAJO, AGUADILLA, PR

1601 SW 89TH ST, UNIT C SUITE 500, OKLAHOMA CITY, OK

CARR 2 KM, 5 BO. CAIIMITAL ALTO, AGUADILLA PR, PR

CARR 140 KM 68, INT. CALLE OBRERO C8, BARCELONETA, PR

115 CENTRAL PARK W, OFC 14, NEW YORK, NY

901 E SUNFLOWER RD, BOLIVAR MEDICAL CTR, CLEVELAND, MS

1000 E EAGER ST, E BALTIMORE MED CTR, BALTIMORE, MD

10228 DUPONT CIRCLE E DR, SUITE 100, FORT WAYNE, IN

4126 S W FWY 600C, NONE, HOUSTON, TX

CARR 100 KM, 1 BARRIO MIRADERO, CABO ROJO, PR

97 W PHILLIP RD, VICTORY CENTRE OF VERNON HILLS, VERNON HILLS, IL

180 W HALF DAY RD, SUNRISE OF BUFFALO GROVE, BUFFALO GROVE, IL

1995 CARR, 2 SUITE 208, BAYAMON, PR

BAYAMON MEDICAL PLAZA, CARR 2 SUITE 104 A, BAYAMON, PR

525 E 68TH ST, RM F9 WEST 9TH FL, NEW YORK, NY

CARR 130 KM, 9 BO CAMPO ALEGRE, HATILLO, PR

CARRETERA 2 KM, SUITE 7, MANATI, PR

221 W COLORADO BLVD, PAV II 525, DALLAS, TX

4955 VAN NUYS BLVD, 415 JABEEN FATIMA MD, SHERMAN OAKS, CA

CARRETERA 686 KM, 4 CABO CAR

1001 N WALDROP DR, 601 ARLINGTON, ARLINGTON, TX

2225 PONCE BY PASS, EDIFICIO PARRA SUITE 606, PONCE, PR

1008 RANCH RD 620, SUITE 200, LAKEWAY, TX

440 W LYNDON B JOHNSON FWY, PLAZA II SUITE 335, IRVING, TX

24 W WASHINGTON ST, BRADFORD MED GRP OF UPMC HAMET, BRADFORD, PA

207 RIVER PARK N DR, NONE, WOODSTOCK, GA

131 OLD RD TO 9 ACRE COR, SUITE 570, CONCORD, MA

4401 WORNALL RD, PEET CENTER 5TH FLOOR, KANSAS CITY, MO

CARR 129 KM 8 SAN LUIS, HOSPITAL PAVIA SUITE 104, ARECIBO, PR

CARR 129 KM, 750 BO HATO ARRIBA, ARECIBO, PR

341 S MARINE CORPS DR RK PLZ, 104 GUAM ENT LLC, TAMUNING, GU

1115 S SUNSET AVE, QUEEN OF VLY HOSP, WEST COVINA, CA

51 W 51ST ST, FL 3 SUITE 360, NEW YORK, NY

255 W SPRING VALLEY AVE, SECOND FLOOR SUITE 200, MAYWOOD, NJ

505 N RIDGEWAY DR, 195 NAOMI R HARMAN MD, CLEBURNE, TX

100 W SPROUL RD, HEALTHPLEX PAVILION II SUITE 100, SPRINGFIELD, PA

HOSPITAL WILMA N VAZQUEZ, CARR 2 KM 39.5 SUITE 105, VEGA BAJA, PR

174 0 CARR 2 KM SAN GERMAN MED PLA, NONE, SAN GERMAN,

1600 S COULTER ST BUILDING G, 501 TOP OF TEXAS ORTHOPEDICS, AMARILLO, TX

HOSP DR CTR TORRE MEDICAL DR PEDRO BLANCO, CALLE 2 KM 47.8 SUITE 208, MANATI, PR

TORRE SAN VICENTE DE PAUL 1ST FL, CARR 2 KM 173, SAN GERMAN, PR

6777 MARGINAL ISLA VERDE, 213 ISLA VERDE MALL, CAROLINA, PR

4640 N MARINE DR 5TH FLOOR BLOOM BLDG, NONE, CHICAGO, IL

4151 JAIME ZAPATA HWY 103, NONE, LAREDO, TX

2851 S AVE B, SUITE 2601, YUMA, AZ

500 W THOMAS RD, 680 DHMG ST JOSEPH'S PELVIC PAIN AND SURG, PHOENIX, AZ

URB VILLA DEL REY 2DA SEC, 2E 1 ESQ BONAPARTE, CAGUAS, PR

CARR 119 KM 5, BARRIO PUENTE SECTOR ZARZA, CAMUY, PR

CARR 110 KM, 3 BARRIO CEIBA BAJA, AGUADILLA, PR

CARR 152 KM, 6 BO. CEDRO ARRIBA, NARANJITO, PR

33 S DELAWARE AVE, 104 B RIVERVIEW PLAZA, YARDLEY, PA

1 TORRE MEDICA, 209 CARR 2, MANATI, PR

1035 S STATE RD, 4 SUITE 120, WELLINGTON, FL

CARR 2 KM, 7 BAYAMON MED PLAZA 810, BAYAMON, PR

1007 W AVE, M14 SUITE B 1, PALMDALE, CA

2138 SCENIC N HWY A, NONE, SNELLVILLE, GA

CARR 64 KM, 5146 BO MA

CARR 152 INT, 803 BO CEDRO ARRIBA, NARANJITO, PR

340 ROUTE 1 N, REDWOOD AVENUE, EDISON, NJ

115 ROUTE 46 WEST BUILDING D, SUITE 27, MOUNTAIN LAKES, NJ

100 CARR, 165 SUITE 310, GUAYNABO, PR

CALLE 16 BLOQUE 35 NUM 25 VILLA, NONE, CAROLINA, PR

111 E 210TH ST, KLAU 1 PAVILION ROOM 25, BRONX, NY

801 ENCINO NEPL, D7 MEDICAL ARTS PLAZA, ALBUQUERQUE, NM

5578 AND 5580 KIRKWOOD HWY, NONE, WILMINGTON, DE

AVE JOSE DE DIEGO, EDIF CHEVERE OFIC 2D4 PDA 22 328, SANTURCE, PR

328 EDIF CHEVERE OFIC 2D4 PDA, SUITE 22, SANTURCE, PR

14904 JEFFERSON DAVIS HWY 306, NONE, WOODBRIDGE, VA

1707 E RED RIVER, DIAGNOSTIC CLINIC OF VICTORIA, VICTORIA, TX

2822 S ALAFAYA TRAIL, 150 CENTRAL FLORIDA EYE CLINIC, ORLANDO, FL

CARR 152 INT 803, SUITE 1, NARANJITO, PR

800 BIESTERFIELD RD, 4009 BROCK BUILDING, ELK GROVE VILLAGE, IL

2225 PONCE BY PASS, EDIFICIO PARRA SUITE 103, PONCE, PR

CARR 931 KM, 0 BO NAVARRO, GURABO, PR

CARR 198 KM, 0 BO MONTONES I, LAS PIEDRAS, PR

22 0 CARR 198 KM BO MONTONES I, NONE, LAS

...

520 E 70TH ST, CORNELL HEMATOLOGY ONCOLOGY, NEW YORK, NY

4048 CEDAR BLUFF DR, 1 BAY STREET ORTHOPAEDICS PLLC, PETOSKEY, MI

6700 W 121ST, 300 GENESIS PAIN CLINIC, OVERLAND PARK, KS

17510 W GRAND PKWY S, 430 SUGAR LAND WOMEN'S CARE, SUGAR LAND, TX

2727 W 2ND ST, 314 CENTRAL NEBRASKA NEUROLOGY, HASTINGS, NE

908 N HOWARD AVE, 108 CHI HEALTH ST FRANCIS SPEC CARE, GRAND ISLAND, NE

908 N HOWARD AVE, 109 GRAND ISLAND SPECIALTY CLINIC, GRAND ISLAND, NE

10 FORBES RD 2ND FL E, NONE, BRAINTREE, MA

12030 SW 171 TERR, NONE, EDELMIRA, FL

1809 NORTHPOINTE LN, 102 QUICKCARE OF RUSTON, RUSTON, LA

170 HEALTHCARE PLAZA DR, MINDEN MEDICAL CLINIC OF HAUGHTON, HAUGHTON, LA

9321 W SUNSET RD, HAND SURGERY SPECIALISTS OF NEVADA, LAS VEGAS, NV

2700 E 29TH ST, 260 CHI ST JOSEPH HEALTH PULMONARY ASSO, BRYAN, TX

535 E 70TH ST, HSS DEPARTMENT OF MEDICINE, NEW YORK, NY

3066 SW MARTIN DOWN S BLVD, UNIT 6 SUITE C, PALM CITY, FL

50 N MEDICAL DR, UNIVERSITY OF UTAH DEPT OF NEUROSURGERY, SALT LAKE CITY, UT


300 EXECUTIVE CTR DR, PALM GARDEN REHAB OF WEST PALM BEACH, WEST PALM BEACH, FL

21020 STATE RD, 7 SUITE 220, BOCA RATON, FL

2960 N STATE RD, 7 SUITE 108, MARGATE, FL

319 ROUTE 130 N 29D, WOMEN'S HEALTH SPECIALISTS OF CENTRASTATE, EAST WINDSOR, NJ

936 STATE ROUTE 160, NONE, GALLIPOLIS, OH

701 N CLAYTON ST, ST FRANCIS HOSPITAL MED SERV BLD SUITE 600, WILMINGTON, DE

321 N FURNACE ST, NOVACARE REHABILITATION 50208, BIRDSBORO, PA

621 S MAIN ST, PENN HIGHLANDS GASTROENTEROLOGY, DU BOIS, PA

621 S MAIN ST, PH ORTHOPAEDICS FRACTURE CLINIC, DU BOIS, PA

994 W HWY 25 70, SUITE 4, NEWPORT, TN

15 -2866 PAHOA VILLAGE RD, BLDG C, PAHOA, HI

1100 S STATE RD, 7 SUITE 104, MARGATE, FL

3347 S STATE RD, 7 SUITE 203, WELLINGTON, FL

10815 RR, 2222 BLDG 3B, AUSTIN, TX

5401 FM, 1626 SUITE 365, KYLE, TX

320 E FONTANERO ST, 201 SIMMONS EYE ASSOCIATES, COLORADO SPRINGS, CO

15805 W MAPLE RD, 104 WEST MAPLE, OMAHA, NE

1395 S STATE RD, 7 SUITE 410, WELLINGTON, FL

3555 HARDEN ST EXT, 15 MEDICAL PARK 

FRAGOSO 4 ES, 12 VILLA FONTANA, CAROLINA, PR

2430 W PIERCE ST, CARLSBAD MEDICAL CTR, CARLSBAD, NM

7097 N EXPRESSWAY, 77 SUITE 5, OLMITO, TX

1801 NE123RD ST, 405 CAUSEWAY SQUARE, NORTH MIAMI, FL

5000 N 26TH ST, 300 MEDEXPRESS URGENT CARE LINCOLN N 26TH ST, LINCOLN, NE

2202N JOHN B DENNIS HWY 200, NONE, KINGSPORT, TN

2500 ENGLISH CREEK RD BLDG 700, 702 ATLANTICARE SURGERY CENTER, EGG HARBOR TWP, NJ

27700 NW FWY, CYPRESS MULTI SPECIALTY SUITE 600, CYPRESS, TX

8300 W CHEYENNE, 106 URGENT CARE EXTRA, LAS VEGAS, NV

4049 I-49 S SERVICE RD, NONE, OPELOUSAS, LA

845 FM 1960 W, 104 GENESIS MEDICAL GROUP, HOUSTON, TX

837 FM 1960 W RD, 107 GENESIS MEDICAL GROUP, HOUSTON, TX

765 N KELLOGG ST, 203 RENALCARE ASSOCIATES SC, GALESBURG, IL

5625 EIGER RD, 220 TEXAS HEART AND VASCULAR, AUSTIN, TX

1253 N VON MINDEN ST, TEXAS HEART AND VASCULAR, LA GRANGE, TX

300 N POINTE CIR, NOVACARE REHABILITAITON 50234, SEVEN FIELDS, PA

2961 W LIBERTY AVE, NOVACARE REHABILITATION 50249, PITTSBURGH, PA

90

41651 FM, 3159 SUITE B, CANYON LAKE, TX

CENTERPLEX BUILDING, 103 CARR 2 KM 133.5, AGUADA, PR

110 WILLIAM ST, NEWARK DEPARTMENT OF HEALTH AND HUM, NEWARK, NJ

3975 I-49 S SERVICES RD, SUITE 200, OPELOUSAS, LA

1600 CRAIN S HWY 401, NONE, GLEN BURNIE, MD

333 COTTMAN AVE, MEDONC ASSOC OF FCCC MGI, PHILADELPHIA, PA

333 COTTMAN AVE, GYN/ONC ASSOC OF FCCC MGI, PHILADELPHIA, PA

333 COTTMAN AVE, PSYCH ASSOC OF FCCC MGI, PHILADELPHIA, PA

333 COTTMAN AVE, GASTRO ASSOC OF FCCC MGI, PHILADELPHIA, PA

333 COTTMAN AVE, PLASTIC/RECON ASSOC OF FCCC MGI, PHILADELPHIA, PA

26908 INDEPENDENCE WAY, 100 LERAY CMNS, EVANS MILLS, NY

1340 WONDER WORLD DR, 2300 BAYLOR SCOTT AND WHITE CLINIC SAN M, SAN MARCOS, TX

1800 S A W GRIMES BLVD, BAYLOR SCOTT AND WHITE CLINIC ROUND, ROUND ROCK, TX

2301 N UNIVERSITY DR, SUITE 211 2ND FLOOR, PEMBROKE PINES, FL

1609 W 40TH AVE, 204 OBGYN ASSOCIATES OF S ARK, PINE BLUFF, AR

1609 W 40TH AVE, 402 OBGYN ASSOCIATES OF S ARK, PINE BLUFF, AR

1609 W 40TH AVE, 201 OBGYN 

2207 S CLEAR RD, 203 ADVTHLTH ENT, KILLEEN, TX

2439 N HARRISON ST, URGENT CARE CENTER OF ARLINGTON, ARLINGTON, VA

1890 W GAUTHIER RD, 145 OBSTETRICS AND GYNECOLOGY OF SOUTHWES, LAKE CHARLES, LA

1890 W GAUTHIER RD, 135 OBSTETRICS AND GYNECOLOGY OF SOUTWE, LAKE CHARLES, LA

142 WALLACE AVE, 201 GATEWAY MEDICAL ASSOCIATE PC, DOWNINGTOWN, PA

3347 STATE RD, 7 SUITE 200, WELLINGTON, FL

175 S UNION BLVD, 315 UCHMG OCCUPATIONAL HEALTH UNION, COLORADO SPRINGS, CO

221 E 29TH ST, 102 UCHMG IM/OCCMED/DM/ENDO ORCHARDS, LOVELAND, CO

11361 N 99TH AVE, 601 WEST VALLEY, PEORIA, AZ

200 EAGLES NE ST DR, SUITE 300D, CANTON, GA

3956 WHISPERING WAY SE, ADDINTON PLACE AT EAST PARIS, GRAND RAPIDS, MI

1175 68TH ST SE, ADDINGTON PLACE OF GRAND RAPIDS, GRAND RAPIDS, MI

100 S MAIN ST, 215 CCHS OB/GYN, SMYRNA, DE

501 W 14TH ST WILMINGTON HOSPITAL, 4154 CCHS CHIEFS SURGICAL SERVICES, WILMINGTON, DE

501 W 14TH ST WILMINGTON HOSPITAL GATEWAY, 4 CCHS DIABETES AND METABOLIC, WILMINGTON, DE

86 OMEGA DR OME

In [50]:
bolos = bolos.fillna('None')
bolo_parsed = parse_bolo_address(bolos)

PO BOX 973, BOX 800, BENNETTSVILLE, SC

200 QUEENS RD STE 400, STE 400, CHARLOTTE, NC

2000 38TH AVE, VERO BEACH FL 32960, VERO BEACH, FL

3492 CHERRYWOOD RD, MALL A SUUITE 230, FLORENCE, SC

2728 SUNSET BLVD STE 201, STE 201, WEST COLUMBIA, SC

601 VAN NESS AVE APT 63, STE 2008, SAN FRANCISCO, CA

1417 PENDLETON RD, CHARLIE NORWOOD VA MEDICAL CENTER, AUGUSTA, GA

8101 CLEARVISTA PKWY STE 185, STE 185, INDIANAPOLIS, IN

206 S MATSON ST, 635 1ST ST N, KERSHAW, SC

1401 DARE RD, DEPT FAM PR, YORKTOWN, VA

1600 7TH AVE S, CHILDRENS HOSPITAL CPP 210, BIRMINGHAM, AL

114 WILLOW AVE, SAN FRANCISCO VA MED CENTER, CORTE MADERA, CA

846 SEAFARER WAY, DEPT OF MENTAL HLTH, CHARLESTON, SC

900 BEACH BLVD, NORTH FLORIDA MEDICAL CLINIC, JAX BCH, FL

8920 WILSHIRE BLVD STE 511, STE 511, BEVERLY HILLS, CA

1517 HAYNESWORTH RD, DORN VA MEDICAL CENTER, COLUMBIA, SC

1 JARRETT WHITE RD, TRIPLER ARMY MED CTR, TRIPLER AMC, HI

89 W COPELAND DR FL 2, STE 100, ORLANDO, FL

1655 BERNARDIN AVE STE 220, STE 220

6458 SUMAC RD, RADIOLOGY ASSOCIATES OF N TEXAS, FORT WORTH, TX

625 N 6TH ST, DIGNITY HEALTH - CANCER INSTITUTE AT ST., PHOENIX, AZ

9750 WINDWATER DR APT 116, STE 130, HOUSTON, TX

4144 N CENTRAL EXPY STE 750, STE 750, DALLAS, TX

107 ASHFORD DR APT 1122, MEDICAL PLAZA STE 2E, WEST MONROE, LA

16406 W PIMA ST, 505 S PLUMMER AVE, GOODYEAR, AZ

2501 CAPEHART RD, 55 MDG/SGHC, OFFUTT AFB, NE

1300 N 12TH ST STE 520, STE 605, PHOENIX, AZ

217 WOLCOTT RD, NEUROLOGY 5TH FLOOR, CHESTNUT HILL, MA

11803 JEFFERSON AVE STE 110, STE 307, NEWPORT NEWS, VA

6560 FANNIN ST STE 1625, STE 1625, HOUSTON, TX

9350 E 35TH ST N STE 103, STE 103, WICHITA, KS

424 PIMENTEL WAY, 501 J ST, SACRAMENTO, CA

PO BOX 212, PO BOX 212, DERBY, VT

4 DEERWOOD AVE NW, 4 DEERWOOD AVE, WADENA, MN

1009 DELTA DR, 3451 LANTANO RD, CORP CHRISTI, TX

4643 WAIMEA CANYON RD, 4643 WAIMEA CANYON DR, WAIMEA, HI

2100 PFINGSTEN RD RM 1078, STE 128, GLENVIEW, IL

5555 E 5TH ST STE 101, STE A110, TUCSON, AZ

15215 ANTLER CREEK DR, M

758 ELKINS LK, 7517 HIGHWAY 75 S, HUNTSVILLE, TX

917 STONE COTTAGE LN, HEALTHCARE ASSOCIATES OF TEXAS, MCKINNEY, TX

1533 S BROWNLEE BLVD STE 100, STE 100, CORP CHRISTI, TX

18685 MAIN ST # A626, STE 108, HUNTINGTN BCH, CA

1303 MCCULLOUGH AVE STE GL70, STE GL70, SAN ANTONIO, TX

1708 ELMEN ST, 1708 ELMEN STREET, HOUSTON, TX

302 W 8TH ST, UNIVERSITY OF TEXAS MEDICAL BRANCH/SKYVI, TYLER, TX

24620 COUNTY ROAD 26, 5214 LINCOLNWAY E, ELKHART, IN

1541 FLORIDA AVE STE 200, STE 200, MODESTO, CA

6630 DE MOSS DR, 11507 ECHO HOLLOW ST., HOUSTON, TX

3417 CAROLINE DR, 1500 EAST MEDICAL CENTER DR, JACKSON, MI

1492 S MILL AVE STE 101, BLDG 1-49 JAMES L DAY ST, TEMPE, AZ

983280 NEBRASKA MEDICAL CTR, UNIV OF NEB MED CTR, OMAHA, NE

3021 KAHAKO PL, MAURICE EWING HALL 2102, KAILUA, HI

2727 W HOLCOMBE BLVD, THIRD FLOOR 3A/SURG DEPT, HOUSTON, TX

5110 HUISACHE ST, LIMITED TO OFFICIAL UNIVERSITY DUTIES ON, BELLAIRE, TX

4500 HILLCREST RD STE 180, STE 205, FRISCO, TX

2400 GOODNIGHT TRL, UNIV OF TX

2202 EXECUTIVE DR STE C, STE 2000, HAMPTON, VA

8839 CLEARWATER CIR, EMER MED RESIDENCY 5TH FL S BLDG, FOGELSVILLE, PA

1867 AIRPORT WAY STE 120B, STE 120B, FAIRBANKS, AK

1975 RIVER RD, 309 CHERRY ST, WILLIAMSBURG, KY

33 OVERLOOK RD STE 311, STE A11, SUMMIT, NJ

A 1302 MEDICAL CTR NORTH, DEPT UROLOGIC SURG, NASHVILLE, TN

1355 PEACHTREE ST NE STE 580, STE 580, ATLANTA, GA

1507 TREDEGAR AVE, CIMS DIVISION, MFL-6W, CATONSVILLE, MD

5721 W 119TH ST, MAIL STOP 2005, OVERLAND PARK, KS

5899 BREMO RD STE 100, STE 100, RICHMOND, VA

1101 KING ST STE 100, STE 100, ALEXANDRIA, VA

221 PARK HILL DR, 3027 OLD CLINIC BUILDING CB, FREDERICKSBRG, VA

2450 ASHBY AVE, DEPARTMENT OF OBSTETRICS AND GYNECOLOGY, BERKELEY, CA

1514 N WAKEFIELD ST, ID CLINIC 2C SOUTH, ROOM 2C-226, ARLINGTON, VA

620 JOHN PAUL JONES CIR, DEPT OF CT SURGERY NAVAL MEDICAL CENTER, PORTSMOUTH, VA

2549 APPLE CREEK LN, DEPT OF OBSTETRICS & GYNECOLOGY, HACIENDA HTS, CA

12127 CLOUDY PEAK LANE, OUTLOOK APTS G 205, SILVERLAKE, WA

6116 DEERBROOK DR, DEPT OF EMERGENCY MEDICINE, NASHVILLE, TN

445 LENOX RD BOX 79, DEPT OF UROLOGY, BROOKLYN, NY

4603 COLISEUM ST, DEPARTMENT OF OBSTETRICS AND GYNECOLOGY, NEW ORLEANS, LA

1213 HERMANN DR STE 470, STE 212, HOUSTON, TX

1278 W EARLY AVE, 7TH FLOOR SOUTH, CHICAGO, IL

19486 OAK GROVE AVE, MR 11326, PRIOR LAKE, MN

1775 PARKER RD SE, ESSENTIAL FAMILY CARE STE C240, CONYERS, GA

6313 LANDFALL DR, UNIVERSITY OF WISCONSIN, MADISON, WI

3959 PLYMOUTH CIR, UNIVERSITY OF WI-MADISON, MADISON, WI

44 BURROWS RD, 600 HIGHLAND AVE, MADISON, WI

1033 N MAYFAIR RD STE 101, STE 1030, WAUWATOSA, WI

9280 W SUNSET RD STE 220, STE 100, LAS VEGAS, NV

2010 ZONAL AVE, DEPARTMENT OF OPHTHALMOLOGY 2ND FLOOR, LOS ANGELES, CA

1300 N VERMONT AVE STE 100, STE 400, LOS ANGELES, CA

1661 SOQUEL DR STE G, STE G, SANTA CRUZ, CA

200 1ST ST SW DEPT NEURO, DEPT OF NEUROLOGY, ROCHESTER, MN

1249 ALICE DR, 8733 WATERTOWN PLANK RD, GREEN BAY, WI

1111 DELAFIELD ST STE 209, STE 209, WAUKESHA, WI

PO BOX

...


13660 N 94TH DR STE D2, STE 112, PEORIA, AZ

3100 SAN PABLO AVE 310, STE A, BERKELEY, CA

30240 HELMANDALE DR, DEPARTMENT OF RADIOLOGY, FRANKLIN, MI

4108 CHURCH POINT RD, CHILDREN'S HOSP OF THE K DAUGH, VIRGINIA BCH, VA

3836 MARBLE RIDGE LN, HOSPITAL MEDICINE ML 9016, MASON, OH

2683 ASHLEY RD, DEPT OF RADIOLOGY / BSH 5056, SHAKER HTS, OH

731 VASSAR ST, 911 VENTURA AVENUE, ORLANDO, FL

533 WESLEY AVE, MC 856 - 1252 CSB, OAK PARK, IL

3 ALUMNI DR STE 302, STE 302, EXETER, NH

833 CHESTNUT ST STE 301, STE 301, PHILADELPHIA, PA

2329 N 59TH ST, 1660 S COLUMBIAN WAY, SEATTLE, WA

19582 BEACH BLVD, UCSF, 505 PARNASSUS AVE, HUNTINGTN BCH, CA

525 BELMONT AVE E APT 8A, STE 4835, SEATTLE, WA

1430 11TH ST NW, 35TH FIGHTER SQUADRON, CANTON, OH

17640 S WOODLAND RD, MAIL CODE L-10, CLEVELAND, OH

9500 GILMAN DR # MC0711, UNIV OF CALIFORNIA SAN DIEGO, LA JOLLA, CA

10595 E RAINTREE DR, NORTH VALLEY EAR, NOSE & THROAT ASSOCIAT, SCOTTSDALE, AZ

PO BOX 357051, QUINTON ROOSEVELT BLVD, SAN DIEGO, 

128 E APPLE ST, CHE BUILDING 2ND FL, DAYTON, OH

804 CENTRAL AVE, SAMARITAN NORTH HLTH CTR, GREENVILLE, OH

182 ROCKINGHAM RD UNIT 9, 182 ROCKINGTON RD STE 9, LONDONDERRY, NH

327 SIGNATURE DR S, 6878 GREYFIELD COURT, BEAVERCRK TWP, OH

8638 OLD TROY PIKE STE 103, STE 103, HUBER HEIGHTS, OH

75 ARCH ST STE B1, STE B1, AKRON, OH

229 W BUTE ST APT 800, STE 2, NORFOLK, VA

960 CLAGUE RD STE 3201, STE 1, WESTLAKE, OH

69355 BLAINE CHERMONT RD, 90 NORTH FOURTH STREET, BRIDGEPORT, OH

502 OAK CREST DR, 125 BAUM ST, KENDALLVILLE, IN

3993 REDFORD CT, 2115 LIGHTER RD, NEW ALBANY, OH

5056 ROCKAWAY LN, DEPT OF EMERGENCY MEDICINE, CLARKSTON, MI

2251 N RAMPART BLVD # 248, NEVADA TASK FORCE 1 - FEMA, LAS VEGAS, NV

770 JASONWAY AVE, JASONWAY INTERNAL MEDICINE-STE G2, COLUMBUS, OH

4700 LAS VEGAS BLVD N, 314 MD/SGH, NELLIS AFB, NV

MEDICAL CENTER BOULEVARD, SECTION OF INFECTIOUS DISEASES, WINSTON SALEM, NC

2587 GARDENIA DR, 582 E WHITT RD, COLUMBUS, OH

4930 W BROAD ST STE 4, STE 1, COLUMBUS, OH

4535 GLEN EDEN RD, 3471 FIFTH AVENUE, CRANBERRY TWP, PA

1684 CANYON OAKS DR, 1000 MEDICAL CENTER DR, MT PLEASANT, SC

36 TEAL DR, 4755 OGLETOWN-STANTON ROAD, LANGHORNE, PA

4 HENDRICKSON LN, SECTION OF INFECTIOUS DISEASES, UNIONVILLE, CT

404 KENT RD, JONES HALL 1ST FLOOR, BALA CYNWYD, PA

2450 VIRGINIA AVE NW APT E102, APT E102, WASHINGTON, DC

6630 S MCCARRAN BLVD STE 8, STE 200, RENO, NV

7500 CENTRAL AVE STE 108, STE 108, PHILADELPHIA, PA

10 UNION SQ E FRNT 2F # 4H, FL 9, NEW YORK, NY

2400 MARYLAND RD STE 20, STE 20, WILLOW GROVE, PA

10 WOODSIDE DR, 2100 CORLIES AVE, RUMSON, NJ

4745 STANTON OGLETWN RD STE 216, STE 216, NEWARK, DE

915 OLD FERN HILL RD # 1B, STE ONE, WEST CHESTER, PA

101 STATES ST APT 6, 4150 CLEMENT ST, SAN FRANCISCO, CA

7 E ORANGEWOOD AVE, 7 EAST ORANGEWOOD AVENUE, PHOENIX, AZ

443 LAUREL OAK RD STE 100, 1ST FL, VOORHEES, NJ

50 E STATE ST, HEALTH SERVICES 3RD FL, TRENTON, NJ

304 PLEASANT DR, 12550 W HIGHWAY 29, AUSTIN, TX

250 PIERCE ST STE 211, STE 211, 

510 PELLIS RD STE 203, STE G10, GREENSBURG, PA

2 FATIMA DR, DEPARTMENT OF NEUROLOGY, ASHLAND, MA

661 MAIN ST, UNIV OF MASS MEDICAL CENTE, SHREWSBURY, MA

1 BIGELOW SQ STE 729, STE 2200, PITTSBURGH, PA

9815 BALMORAL CIR, DEPT OF EMERGENCY MEDICINE, CHARLOTTE, NC

236 COE RD, MERCY HOSP MED CTR, CLARENDON HLS, IL

1015 SOPHIA LN, DEPARTMENT OF EMERGENCY MEDICINE, MARS, PA

1849 KEISER RIDGE RD, LEE STREET 1ST FLOOR, CHARLOTTESVLE, VA

523 3RD AVE, 2205 2ND AVE W, INTL FALLS, MN

2301 M ST NW, WEST END MEDICAL CENTER, WASHINGTON, DC

5000 SAGEMORE DR STE 103, STE 103, MARLTON, NJ

21 W RUNSWICK DR, 1200 E BROAD ST, HENRICO, VA

3619 PARK EAST DR STE 303, STE 206, BEACHWOOD, OH

120 E NORTH AVE, DEPARTMENT OF RADIOLOGY, PITTSBURGH, PA

21 OJIBWA CIR, CENTER 10TH FLOOR, BUFFALO, NY

7209 MED CTR E S TWR, VANDERBILT UNIV MED CTR, NASHVILLE, TN

701 GROVE RD, SUPPORT TOWER 5TH FLOOR, GREENVILLE, SC

3266 N MERIDIAN ST STE 606, STE 202, INDIANAPOLIS, IN

250 FAME AVE STE 204, STE 104, HANOV

11.7 SUITE 502 CARR # 2 KM, BAYAMON MEDICAL PLAZA, BAYAMON, PR

2617 NE 23RD ST, ACC E RM #352 (D48), FT LAUDERDALE, FL

66 CALLE SANTA CRUZ, PLAZA SAN PABLO I SUITE 101, BAYAMON, PR

CAROLINA SHOPP CTR, MULTIPISO OF 309 AVE 65 INFANTERIA, CAROLINA, PR

21 CHRISMAN RD, PISOS DE CAPARRA 14 CALLE MILAN, FORT BUCHANAN, PR

J16 CALLE 2, B EDIFICIO MEDICO HERMANAS DAVILA 16 CAL, BAYAMON, PR

3621 S STATE ST, UNIVERSITY OF MICHIGAN DEPRESSION CENTER, ANN ARBOR, MI

4863 SCOTTSVILLE RD STE B, STE 301, BOWLING GREEN, KY

3880 INVERNESS WAY, DEPT OF PEDIATRICS BG 2128, AUGUSTA, GA

SOUTH WING PO BOX 365067, GAMMA PROJECT ACTU 4TH FLO, SAN JUAN, PR

919 PEACHTREE BATTLE CIR NW, DEPARTMENT OF OBSTETRICS & GYNECOLOGY, ATLANTA, GA

CARR 734 KM 0 5 BO ARENAS, 0 5 CARR 734, CIDRA, PR

4900 W OKLND PRK BLVD STE 101, STE 405, LAUD LAKES, FL

PO BOX 1036, MEZZANINE SUITE B, GUAYNABO, PR

PO BOX 506, 2 STATE RD OFFICE 808, BAYAMON, PR

1065 AVE LOS CORAZONES, # OFIC 201, MAYAGUEZ, PR

1010 4TH ST SW STE 

In [51]:
gov_all_parsed = pd.merge(gov, gov_parsed, on='adrs_id')
bolos_all_parsed = pd.merge(bolos, bolo_parsed, on='comm_id').drop_duplicates('ME')

In [52]:
just_a_test = pd.merge(bolos_all_parsed, gov_all_parsed, on='ME', suffixes = ['_BOLO','_GOV'])

In [53]:
just_a_test = just_a_test.fillna('None')

In [57]:
dicts =[]
for row in just_a_test.itertuples():
    MATCH = False
    if row.addr_state == row.st:
        if is_a_match(row.StreetName_GOV, row.StreetName_BOLO,80):
            MATCH = True
        elif row.ZIP_GOV == row.ZIP_BOLO and is_a_match(row.adr_ln_1, row.addr_line_2, 75):
            MATCH = True
        if MATCH == True:
            score = fuzz.ratio(row.adr_ln_1, row.addr_line_2)
            print(f'{row.adr_ln_1} is {row.addr_line_2}')
            new_dict = {
                'ME': row.ME,
                'adrs_id': row.adrs_id,
                'comm_id': row.comm_id,
                'GOV_ADDR': row.adr_ln_1,
                'BOLO_ADDR': row.addr_line_2,
                'GOV_ZIP': row.ZIP_GOV,
                'BOLO_ZIP':row.ZIP_BOLO,
                'SCORE': score,
                'PHONE': row.PHONE
                }
            dicts.append(new_dict)

2525 COURT DR is 2525 COURT DR
766 WALTHER RD is 755 WALTHER RD
2000 38TH AVE is 2000 38TH AVE
1825 MARTHA BERRY BLVD NW is 1104 MARTHA BERRY BLVD NE
702 N A ST is 2 INNOVATION DR STE 400
2 INNOVATION DR is 2 INNOVATION DR STE 400
330 HOSPITAL DR is 330 HOSPITAL DR
2821 ALTERNATE US 27 is 2821 ALT US HWY 27 S
2410 W PIERCE ST is 2428 W PIERCE ST
2428 W PIERCE ST is 2428 W PIERCE ST
2428 W PIERCE ST is 2428 W PIERCE ST
1519 MARION ST is 1519 MARION ST
1519 MARION ST is 1519 MARION ST
1541 KINGS HWY is 1501 KINGS HWY
1606 KINGS HWY is 1501 KINGS HWY

83
------------------------------------
1 RICHLAND MEDICAL PARK DR is 1 RICHLAND MED PK DR STE 300

83
------------------------------------
1 RICHLAND MEDICAL PARK DR is 1 RICHLAND MED PK DR STE 300
5325 APPIAN WAY is 5325 APPIAN WAY
2361 CYPRESS CIR is 2361 CYPRESS CIR
2000 E GREENVILLE ST is 2000 E GREENVILLE ST STE 1600
5304 INDIAN GRAVE RD is 5303 INDIAN GRAVE RD
115 N SUMTER ST is PO BOX 1469
115 N SUMTER ST is PO BOX 1469
485 HWY 29 N 

110 W SQUANTUM ST is 110 W SQUANTUM ST
2490 CENTRAL AVE is 2490 CENTRAL AVE
210 25TH AVE N is 210 25TH AVE N
1111 SHADOW LN is 1111 SHADOW LN
2005 TECHNOLOGY PKWY is 2005 TECHNOLOGY PKWY
2015 TECHNOLOGY PKWY is 2005 TECHNOLOGY PKWY
2005 TECHNOLOGY PKWY is 2005 TECHNOLOGY PKWY

97
------------------------------------
1209 A MARDA LN is 1209A MARDA LN
20 MAYO RD is 1209A MARDA LN
600 N HIGHLAND SPRINGS AVE is 600 N HIGHLAND SPRINGS AVE
1 GOOD SAMARITAN WAY is 2 GOOD SAMARITAN WAY STE 205
2 GOOD SAMARITAN WAY is 2 GOOD SAMARITAN WAY STE 205
747 BROADWAY is 600 BROADWAY
2500 W 12TH ST is 2500 W 12TH ST
16651 SOUTHWEST FWY is 16651 SOUTHWEST FWY
590 SPRINGRIDGE RD is 590 SPRINGRIDGE RD
512 N SHADY LN is 512 N SHADY LN
1725 CAPITAL N E CIR is 1725 CAPITAL CIR NE STE 305
600 N WOLFE ST is 600 N WOLFE ST
600 N WOLFE ST is 600 N WOLFE ST
600 N WOLFE ST is 600 N WOLFE ST
600 N WOLFE ST is 600 N WOLFE ST
600 N WOLFE ST is 600 N WOLFE ST
600 N WOLFE ST is 600 N WOLFE ST
2450 ASHBY AVE is 2450 ASHB

8220 WALNUT HILL LN is 8220 WALNUT HILL LN STE 408
8200 WALNUT HILL LN is 8220 WALNUT HILL LN STE 408
909 9TH AVE is 3810 139TH ST
6560 FANNIN ST is 6550 FANNIN ST
3201 HWY 380 101 is 3201 US HIGHWAY 380 STE 101
3417 GASTON AVE is 3417 GASTON AVE STE 830
5200 HARRY HINES BLVD is 5323 HARRY HINES BLVD
1601 W HEBRON PKWY is 1601 W HEBRON PKWY
1455 E BERT KOUN LOOP is 1400 E BERT KOUN LOOP
3501 CATTLEMEN RD is 3501 CATTLEMEN RD
102 E YOUNG ST is 102 E YOUNG ST
1662 HIGDON FERRY RD is 1662 HIGDON FERRY RD STE 200
6565 FANNIN ST is 6560 FANNIN ST STE 1008
6565 FANNIN ST MAIN is 6560 FANNIN ST STE 1008
6550 FANNIN ST is 6560 FANNIN ST STE 1008
1201 SUMMIT AVE is 1201 SUMMIT AVE
2500 W STRUB RD is 2500 W STRUB RD STE 210
935 HIGHLAND BLVD is 935 HIGHLAND BLVD STE 3220
11442 N CENTRAL EXPY is 11442 N CENTRAL EXPY
500 UNIVERSITY AVE is 500 UNIVERSITY AVE
2000 S MAYS ST is 2000 S MAYS ST
2000 S MAYS ST is 2000 S MAYS ST
3850 G ST is 4230 WINDING CREEK RD
1400 W 4TH ST is 1400 W 4TH ST

86
------

5410 W S LOOP is 4342 SAN CARLOS ST
200 E EXPRESSWAY is 3831 KEY HOLLOW WAY
2501 WILLIAM CANNON DR is 2501 W WILLIAM CANNON DR
2501 W WILLIAM CANNON DR is 2501 W WILLIAM CANNON DR

89
------------------------------------
987400 NEBRASKA MEDICAL CTR is 983280 NEBRASKA MEDICAL CTR
2000 CENTRE POINTE BLVD is 2000 CENTRE POINTE BLVD
2000 CENTRE POINTE BLVD is 2000 CENTRE POINTE BLVD
2000 CENTRE POINTE BLVD is 2000 CENTRE POINTE BLVD
9225 KATY FWY is 9225 KATY FWY
6161 S YALE AVE is 6151 S YALE AVE
6600 S YALE AVE is 6151 S YALE AVE
15055 E FWY is 15055 EAST FWY STE B60
919 E 32ND ST is 919 E 32ND ST
400 N PEPPER AVE is 400 N PEPPER AVE STE 308
6400 FANNIN ST is 6400 FANNIN ST STE 3000
4101 JAMES CASEY ST is 4101 JAMES CASEY ST STE 100
1301 S COULTER ST is 1301 S COULTER ST STE 413
14700 E OLD US HWY 12 is 14700 E OLD US HIGHWAY 12
800 ORTHOPEDIC WAY is 800 ORTHOPEDIC WAY
1441 EASTLAKE AVE is 1441 EASTLAKE AVE STE 244
3601 HOUMA BLVD is 3901 HOUMA BLVD
1525 LOS ALTOS PKWY is 1525 LOS ALTOS 


92
------------------------------------
209 MARTN LTHR KING JR WAY is 209 MARTIN LUTHER KING JR WAY
1233 LAWRENCE ST is 1334 LAWRENCE ST

89
------------------------------------
322 W N RIVER DR is 322 W NORTH RIVER DR
2 BON AIR RD is 2 BON AIR RD STE 100
4011 TALBOT RD S is 4011 TALBOT RD S STE 300
1003 PROVIDENCE DR is 1003 N PROVIDENCE DR
130 BRENTWOOD DR is PO BOX 1000
5610 W GAGE is 5610 W GAGE ST
5350 TALLMAN AVE NW is 5350 TALLMAN AVE NW STE 301
1300 N MAIN ST is 1300 N MAIN ST
55 W TIETAN ST is 55 W TIETAN ST
1231 116TH AVE NE is 1231 116TH AVE NE
21632 HWY 99 is 21632 HIGHWAY 99
2601 E ROOSEVELT ST is 2601 E ROOSEVELT ST
1400 E BOULDER ST is 1725 E BOULDER ST
621 W MADRONE ST is 621 W MADRONE ST
1229 MADISON ST is 1229 MADISON ST STE 750
1229 MADISON ST is 1229 MADISON ST STE 750
1229 MADISON ST is 1229 MADISON ST STE 750
525 LILLY RD NE is 525 LILLY RD NE STE 210
311 S L ST is 7917 ZANGLE RD NE
212 NESKYLINE DR is PO BOX 1519
37 S 2ND is 37 S 2ND E
11960 LIONESS WAY is 11960

...

300 COMMUNITY DR is 300 COMMUNITY DR DEPT RADI
330 COMMUNITY DR is 300 COMMUNITY DR DEPT RADI
350 COMMUNITY DR is 300 COMMUNITY DR DEPT RADI
300 COMMUNITY DR LL is 300 COMMUNITY DR DEPT RADI
6485 DAY ST is 6485 DAY ST
3527 SACRAMENTO ST is 3527 SACRAMENTO ST
279 TROY RD is 279 TROY RD - RT4
237 RICHMOND VALLEY RD is 237 RICHMOND VALLEY RD
710 N BEAVER ST is 710 N BEAVER ST BLDG 3
101 THE CITY DR S is 101 THE CITY DR S
530 1ST AVE is 530 1ST AVE STE 84
550 1ST AVE is 530 1ST AVE STE 84
2800 L ST is 8170 LAGUNA BLVD
160 E HANOVER AVE is PO BOX 1446

93
------------------------------------
1 GUSTAVE L LEVI PL is BOX 1030
1740 W TAYLOR ST is 1740 W TAYLOR ST
14520 W GRANITE VALLEY DR is 14520 W GRANITE VALLEY DR
400 PARNASSUS AVE is 500 PARNASSUS AVE BOX 136
505 PARNASSUS AVE is 500 PARNASSUS AVE BOX 136
2515 NORTHBROOKE PLAZA DR is 2515 NORTHBROOKE PLAZA DR
150 WHITE PLAINS RD is 150 WHITE PLAINS RD
2730 S VAL VISTA DR is 2730 S VAL VISTA DR
95 S PAGOSA BLVD is 95 S PAGOSA BLVD
1245 WILSH

9961 SIERRA AVE is 9985 SIERRA AVE
311 N ST is 664 STONELEIGH AVE
11800 WILSHIRE BLVD is 11800 WILSHIRE BLVD FL 3
350 N WILMOT RD is 677 N WILMOT RD
677 N WILMOT RD is 677 N WILMOT RD
330 N WILMOT RD is 677 N WILMOT RD
710 LAWRENCE EXPRESS WAY is 710 LAWRENCE EXPY
700 LAWRENCE EXPY is 710 LAWRENCE EXPY
1680 ROUTE 23 N is 1680 ROUTE 23
3692 HIGHLANDS SEPKWY is 3692 HIGHLANDS PKWY SE
1001 POTRERO AVE is 1001 POTRERO AVE RM 5H16
202 S PARK ST is 20 S PARK ST
1 S PARK ST is 20 S PARK ST
20 S PARK ST is 20 S PARK ST
161 FORT WASHINGTON AVE is 161 FORT WASHINGTON AVE 12TH FL
6 SAND HILL RD is 6 SAND HILL RD
4 HUNTER ST is 4 HUNTER ST
5 E 98TH ST 2ND FL is 5 E 98TH ST FL 7
2808 OLD POST RD is 2808 OLD POST RD
32270 TELEGRAPH RD is 32270 TELEGRAPH
3700 JOSEPH SIEWICK DR is 3700 JOSEPH SIEWICK DR
332 SHAWMUT AVE is 332 SHAWMUT AVE
600 PLAZA CT is 600 PLAZA CT
243 CHARLES ST is 243 CHARLES ST 2ND FL
136 MOUNTAINVIEW BLVD is 136 MOUNTAINVIEW BLVD
530 1ST AVE is 530 1ST AVE
550 1ST AVE is 530 1ST 

1200 12TH AVE S is 1200 12TH AVE S
466 OLD HOOK RD is 466 OLD HOOK RD
250 OLD HOOK RD is 466 OLD HOOK RD
1801 ASHLEY CIR is 1801 ASHLEY CIR
1791 ASHLEY CIR is 1801 ASHLEY CIR
1577 CONGRESS ST is 1577 CONGRESS ST
2123 AUBURN AVE is 2123 AUBURN AVE # 724
2383 PATE ST N is 2383 PATE ST N
833 CHESTNUT ST is 833 CHESTNUT ST STE 301
4 DEERWOOD AVE NW is 4 DEERWOOD AVE NW
23625 HOLMAN HWY is 23625 HOLMAN HWY
4725 -4733 SUNSET BLVD is 4950 W SUNSET BLVD 6TH FL
4733 W SUNSET BLVD is 4950 W SUNSET BLVD 6TH FL
4950 W SUNSET BLVD is 4950 W SUNSET BLVD 6TH FL
10500 MONTGOMERY RD is 4313 N MASON MONTGOMERY RD
30420 HAUN RD is 30420 HAUN RD
6482 E MAIN ST is 6482 E MAIN ST
5881 W 16TH ST is 5881 W 16TH ST
6105 1ST FINANCIAL DR is 6159 1ST FINANCIAL DR
8330 LAKEWOOD RANCH BLVD is 8340 LAKEWOOD RANCH BLVD
400 MATTHEW ST is 400 MATTHEW ST
400 MATTHEW ST is 400 MATTHEW ST
2999 INNOVATION WAY is 2999 INNOVATION WAY
20 W WENGER RD is 20 W WENGER RD
437 PORTAGE TRL is 437 PORTAGE TRL
437 PORTAGE TRAIL is 43

9701 SW BARNES is 9701 SW BARNES RD
2955 IVY RD is 2955 IVY RD STE 304

81
------------------------------------
10180 SE SUNNYROAD RD is 10210 SE SUNNYSIDE RD
10100 SE SUNNYSIDE RD is 10210 SE SUNNYSIDE RD
269 MAPLE ST is 269 MAPLE ST
1531 ESPLANADE is 1531 ESPLANADE
10100 SE SUNNYSIDE RD is 9900 SE SUNNYSIDE RD
774 CHRISTIANA RD is 774 CHRISTIANA RD
915 LAWN AVE is 915 LAWN AVE
530 N MAIN ST is 530 N MAIN ST
2004 BREMO RD is 2004 BREMO RD STE 201
401 N 17TH ST is 401 N 17TH ST STE 312
4 BROTHERTON WAY is 4 BROTHERTON WAY
49 NESCONSET HWY is 2500 NESCONSET HWY
2500 NESCONSET HWY is 2500 NESCONSET HWY
419 N HARRISON ST is 419 N HARRISON ST
2580 HAYMAKER RD is 2580 HAYMAKER RD
949 CENTRAL AVE is 949 CENTRAL AVE STE 102
123 FRANKLIN CORNER RD is 123 FRANKLIN CORNER RD
6649 CURLEW TER is 6649 CURLEW TER
925 CHESTNUT ST is 925 CHESTNUT ST FL 6
600 HIGHLAND AVE is 600 HIGHLAND AVE
2004 SPROUL RD is 2004 SPROUL RD
741 W MARLTON PIKE is 741 MARLTON PIKE W
8322 BELLONA AVE is 8322 BELLONA AVE
8

21316 39TH AVE is 21316 39TH AVE
130 LA CASA VIA is 130 LA CASA VIA STE 209
65 W JIMMIE LEEDS RD is 72 W JIMMIE LEEDS RD STE 1100
44 E JIMMIE LEEDS RD is 72 W JIMMIE LEEDS RD STE 1100
7200 PEACH ST is 3330 PEACH ST STE 106
5035 PEACH ST is 3330 PEACH ST STE 106
915 OLD FERN HILL RD A is 915 OLD FERN HILL RD
5 FEDERAL ST is 5 FEDERAL ST
2729 BLAIR MILL RD C NORTHWOOD BLDG is 2701 BLAIR MILL RD
501 DISCOVERY DR is 501 DISCOVERY DR
3231 S EUCLID is 3231 EUCLID AVE
3231 EUCLID AVE is 3231 EUCLID AVE
1400 LOCUST ST is 1350 LOCUST ST
1400 LOCUST ST is 1350 LOCUST ST
1350 LOCUST ST is 1350 LOCUST ST
3222 SHRINE RD A is 3222 SHRINE RD STE A
830 OLD LANCASTER RD is 830 OLD LANCASTER RD STE 206
3970 W HWY 27 is 4429 W HIGHWAY 27
4429 W HWY 27 is 4429 W HIGHWAY 27
14 ELLIOT AVE is 14 ELLIOTT AVE
320 E N AVE is 1320 MERIDIAN DR
171 FAIRVIEW RD is 171 FAIRVIEW RD
1365 WASHINGTON AVE is 220 WASHINGTON AVENUE EXT
27 S 5TH AVE is 27 S 5TH AVE
904 SAHARA TRL is 904 SAHARA TRL
4700 SPRING ST is 4700 SPR

In [58]:
CHECK = pd.DataFrame(dicts).sort_values('SCORE').drop_duplicates('ME', keep='last')

In [69]:
len(CHECK)

11282

In [54]:
pd.merge(bolos_all_parsed, gov_all_parsed, on=['ME', 'ZIP'], suffixes = ['_BOLO','_GOV']).drop_duplicates('ME')[['adr_ln_1','addr_line2']]

,adr_ln_1,addr_line2
0,2525 COURT DR,2525 COURT DR
3,1255 VISCAYA PKWY,1255 VISCAYA PKWY
4,935 W 2ND ST,935 W SECOND ST
5,815 OLD CHEROKEE RD,134 SHARON LAKE CT
7,330 HOSPITAL DR,330 HOSPITAL DR
...,...,...
15840,1721 EBENEZER RD,1721 EBENEZER RD STE 175
15841,2021 N MYRTLE POINT BLVD,2021 N MYRTLE POINT BLVD
15842,815 OLD CHEROKEE RD,116 TURNBERRY LN
15843,14 EDGEWOOD DR,14 EDGEWOOD DR


In [76]:
NEW_PHONES = pd.merge(CHECK[CHECK.PHONE=='None'], phoney, on=['ME','adrs_id']).drop_duplicates('ME')

In [75]:
CHECK[(CHECK.PHONE=='None')&(CHECK.adrs_id.isin(phoney.adrs_id)==False)]

,ME,adrs_id,comm_id,GOV_ADDR,BOLO_ADDR,GOV_ZIP,BOLO_ZIP,SCORE,PHONE
14038,04201880179,PR009350000RIADMINDEXX602,5278673,ADMINISTRACION DE SERVICIOS MEDICOS (MANEJO DE,PO BOX 1036,00935,00970,7,None
697,04815920576,TX769035919SA120XXAVEX400,7599025,120 E BEAUREGARD AVE,PO BOX 22000,76903,76902,12,None
2311,64935900032,PR007250000CAAVEXXMARI409,6053303,AVE LUIS MUNOZ MARIN,PO BOX 4952,00725,00726,13,None
14035,04201841173,PR006120000AR2XXXXRDXX200,2020959,2 RD,PO BOX 9429,00612,00960,13,None
3875,04201031104,PR009350000RIADMINMEDI403,19581655,ADMINISTRACION DE SERVICIOS MEDICOS,PO BOX 1177,00935,00954,13,None
822,05002990280,MT598024003MI500XXBROA301,10755055,500 W BROADWAY,2711 PINNACLE PL,59802,59808,13,None
1560,40910830033,WI532264874MI9000XAVNU400,7823127,9000 W WISCONSIN AVNUE,1451 PARISH DR,53226,53033,22,None
2615,84705740428,PR007913222HUFONTX358X301,3399380,FONT MARTELO 358,PO BOX 8987,00791,00792,22,None
2783,00401940383,AR727123208BE2717XSTXX402,6467956,2717 SE I ST,300 N CLIFTON ST,72712,71742,43,None
11640,01175101281,FL327784359TABESTXLANE600,9051618,BEST AMERICAN STORAGE 1940 NIGHTINGALE LANE,1876 NIGHTINGALE LN,32778,32778,52,None


In [77]:
NEW_PHONES['PHONE']=NEW_PHONES.phn_numbr

In [79]:
maybe = pd.concat([CHECK[CHECK.PHONE!='None'],NEW_PHONES[CHECK.columns]])

In [80]:
pd.merge(bolos, maybe, on='ME').sort_values('SCORE')[0:20]

,entity_id,me#,comm_id_x,usage,load_type_ind,addr_type,addr_line_1,addr_line_3,addr_line_2,addr_city,...,source_dtm,ME,adrs_id,comm_id_y,GOV_ADDR,BOLO_ADDR,GOV_ZIP,BOLO_ZIP,SCORE,PHONE
2460,1646017,506952476,18056427,PO,R,OF,None,None,PO BOX 8697,RCHO SANTA FE,...,2021-08-25 13:26:27.136627,00506952476,CA920141992DE2600XVALL502,18056427,2600 VIA DE LA VALLE,PO BOX 8697,92014,92067,6,8.58499e+09
3199,22167050,5104050516,8041187,PO,R,OF,None,1223 E MARSHALL ST,PO BOX 980677,RICHMOND,...,2021-08-25 13:26:27.136627,05104050516,VA232985051RI1250XSTXX400,8041187,1250 E MARSHALL ST,PO BOX 980677,23298,23298,6,8.04829e+09
9214,5505574,2501782541,7690913,PO,R,OF,None,5620 WILLIAM PENN HWY,PO BOX 185,MURRYSVILLE,...,2021-08-25 13:26:27.136627,02501782541,PA156329036MU5620XHWYX401,7690913,5620 WILLIAM PENN HWY,PO BOX 185,15632,15668,6,7.24328e+09
10484,8221716,3607930626,8150184,PO,R,OF,PROVIDENCE ALASKA MEDICAL,3200 PROVIDENCE DR,PO BOX 196604,ANCHORAGE,...,2021-08-25 13:26:27.136627,03607930626,AK995084615AN3200XDRXX300,8150184,3200 PROVIDENCE DR,PO BOX 196604,99508,99519,6,9.07562e+09
11086,9770549,4177791836,7022184,PO,R,OF,None,1738 W CHELTENHAM AVE,PO BOX 14299,PHILADELPHIA,...,2021-08-25 13:26:27.136627,04177791836,PA191261546PH1738XAVEX400,7022184,1738 W CHELTENHAM AVE,PO BOX 14299,19126,19138,6,2.15548e+12
5598,39075136,1001101184,19914366,PO,R,OF,None,1 GUSTAVE L LEVY PL,BOX 1010,NEW YORK,...,2021-08-25 13:26:27.136627,01001101184,NY100296504NE1XXXXPLXX500,19914366,1 GUSTAVE L LEVI PL,BOX 1010,10029,10029,7,2.12242e+09
4917,29359599,15408050046,19896810,PO,R,OF,None,1 GUSTAVE L LEVY PL,BOX 1230,NEW YORK,...,2021-08-25 13:26:27.136627,15408050046,NY100296500NE1XXXXPLAC400,19896810,1 GUSTAVE LEVY PLACE,BOX 1230,10029,10029,7,2.12732e+09
3613,2632938,1102840581,15908731,PO,R,OF,EMORY UNIVERSITY HOSPITAL,1364 CLIFTON RD NE,ANESTHESIOLOGY / EMORY UNIV HOSPITAL,ATLANTA,...,2021-08-25 13:26:27.136627,01102840581,GA303221013AT1365XNEXX400,15908731,1365 CLIFTON RD NE,ANESTHESIOLOGY / EMORY UNIV HOSPITAL,30322,30322,7,4.04779e+09
9909,7039639,3503930152,20419456,PO,R,OF,MOUNT SINAI SCHOOL OF MEDICINE,1 GUSTAVE L LEVY PL,BOX 1030,NEW YORK,...,2021-08-25 13:26:27.136627,03503930152,NY100296504NE1XXXXPLXX500,20419456,1 GUSTAVE L LEVI PL,BOX 1030,10029,10029,7,2.12732e+09
1467,13652050,65401830086,9050670,PO,R,OF,None,1225 W A DR,PO BOX 2410,LARGO,...,2021-08-25 13:26:27.136627,65401830086,FL337108529SA6133XAVEX300,9050670,6133 CENTRAL AVE,PO BOX 2410,33710,33779,7,7.27344e+09


In [81]:
maybe['phone_number'] = [use.fix_phone(x) for x in maybe.PHONE]

In [101]:
BOLOS.columns

Index(['entity_id', 'me#', 'comm_id', 'usage', 'load_type_ind', 'addr_type',
       'addr_line_1', 'addr_line_3', 'addr_line_2', 'addr_city', 'addr_state',
       'addr_zip', 'addr_plus4', 'addr_country', 'source', 'source_dtm', 'ME'],
      dtype='object')

In [102]:
pd.merge(BOLOS, maybe, on=['ME','comm_id'], how='left')[['entity_id', 'me#', 'comm_id', 'usage', 'load_type_ind', 'addr_type',
       'addr_line_1', 'addr_line_3', 'addr_line_2', 'addr_city', 'addr_state',
       'addr_zip', 'addr_plus4', 'addr_country', 'source', 'source_dtm', 'phone_number']].to_csv('../../Data/PhoneAppend/ADDRESS_SCORING_BATCH_2021-08-25_PHONE_APPEND.txt', index=None, sep='|')

In [85]:
yeah = pd.merge(bolos, maybe, on=['ME','comm_id'], how='left')[['entity_id', 'me#', 'comm_id', 'usage', 'load_type_ind', 'addr_type',
       'addr_line_1', 'addr_line_2', 'addr_line_3', 'addr_city', 'addr_state', 'addr_zip',
       'addr_plus4', 'addr_country', 'source', 'source_dtm', 'phone_number']]

In [89]:
YEAH = yeah.fillna('None')

In [90]:
YEAH[YEAH.phone_number!='None']

,entity_id,me#,comm_id,usage,load_type_ind,addr_type,addr_line_1,addr_line_2,addr_line_3,addr_city,addr_state,addr_zip,addr_plus4,addr_country,source,source_dtm,phone_number
23,10004809,4501830637,7932210,PO,R,OF,None,2000 38TH AVE,VERO BEACH FL 32960,VERO BEACH,FL,32960,None,None,ASM,2021-08-25 13:26:27.136627,7727942227
26,10005394,4501831145,2068360,PO,R,OF,None,1104 MARTHA BERRY BLVD NE,STE E,ROME,GA,30165,None,None,ASM,2021-08-25 13:26:27.136627,7622352495
30,10005719,4501831404,12409230,PO,R,OF,None,2 INNOVATION DR STE 400,UPSTATE CARDIOLOGY PA,GREENVILLE,SC,29607,None,None,ASM,2021-08-25 13:26:27.136627,8642357665
48,10008293,4501850026,25605540,PO,R,OF,"COLISEUM PRIMARY CARE SERVICES, LLC",330 HOSPITAL DR,BLDG A,MACON,GA,31217,None,None,ASM,2021-08-25 13:26:27.136627,4787508606
49,10008332,4501850069,15242121,PO,R,OF,None,2821 ALT US HWY 27 S,MARK ASHBY MD PA,SEBRING,FL,33870,None,None,ASM,2021-08-25 13:26:27.136627,8633823914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101527,9985010,4501731175,6225353,PO,R,OF,None,68 HOSPITAL RD,BLDG C,SYLVA,NC,28779,None,None,ASM,2021-08-25 13:26:27.136627,8285867164
101536,9987571,4501741936,1741541,PO,R,OF,None,303 E PAR ST,None,ORLANDO,FL,32804,None,None,ASM,2021-08-25 13:26:27.136627,8778763627
101547,9990574,4501761295,23147187,PO,R,OF,None,51 NASSAU ST,FETTER HEALTHCARE NETWORK,CHARLESTON,SC,29403,None,None,ASM,2021-08-25 13:26:27.136627,8437224112
101550,9991328,4501770111,8337958,PO,R,OF,None,171 ASHLEY AVE,MUSC ORTHOPAEDICS,CHARLESTON,SC,29425,None,None,ASM,2021-08-25 13:26:27.136627,8437921414


In [91]:
yeah

,entity_id,me#,comm_id,usage,load_type_ind,addr_type,addr_line_1,addr_line_2,addr_line_3,addr_city,addr_state,addr_zip,addr_plus4,addr_country,source,source_dtm,phone_number
0,10000103,4501810326,5711907,PO,R,OF,LEXINGTON CO HLTH SVS DIST INC,627 SPRING LAKE RD,RADIOLOGY DEPARTMENT,COLUMBIA,SC,29206,None,None,ASM,2021-08-25 13:26:27.136627,NaN
1,10000233,4501810423,2065529,PO,R,OF,None,2 MEMORIAL MEDICAL DR,None,GREENVILLE,SC,29605,None,None,ASM,2021-08-25 13:26:27.136627,NaN
2,10000324,4501810491,6225203,PO,R,OF,None,2525 COURT DR,SOMERSET HLTH & WELLNESS C,GASTONIA,NC,28054,None,None,ASM,2021-08-25 13:26:27.136627,NaN
3,10000363,4501810521,11466799,PO,R,OF,None,1130 PERRY HWY,STE 41,PITTSBURGH,PA,15237,None,None,ASM,2021-08-25 13:26:27.136627,NaN
4,10000415,4501810571,9098975,PO,R,OF,None,130 LAURENT WAY,STE A,IRMO,SC,29063,None,None,ASM,2021-08-25 13:26:27.136627,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101602,9999284,4501801475,6584398,PO,R,OF,None,126 COUNTRY CLUB DR,GREENWOOD UROLOGICAL,GREENWOOD,SC,29646,None,None,ASM,2021-08-25 13:26:27.136627,NaN
101603,9999609,4501801769,6348109,PO,R,OF,"24ON PHYSICIANS, PC",940 HOLLY ST,THE REGIONAL MEDICAL CENTER,ORANGEBURG,SC,29115,None,None,ASM,2021-08-25 13:26:27.136627,NaN
101604,9999713,4501801882,6452871,PO,R,OF,None,861 DENALI DR,None,CONWAY,SC,29526,None,None,ASM,2021-08-25 13:26:27.136627,NaN
101605,9999804,4501810075,15817052,PO,R,OF,None,PO BOX 621749,G. STEVEN AVIDON MD PA,ORLANDO,FL,32862,None,None,ASM,2021-08-25 13:26:27.136627,NaN


In [103]:
BOLOS = pd.read_csv('../../Data/PhoneAppend/HSG_PHYS_BOLO_Address_2021-08-25.txt', sep='|')
BOLOS['me#'] = use.fix_me(BOLOS['me#'])

In [105]:
BOLOS['ME'] = use.fix_me(BOLOS['me#'])

In [99]:
pd.merge(CHECK, just_a_test, on=['ME','adrs_id','comm_id'])[['addr_line_3','addr_line_2','StreetName_GOV', 'StreetName_BOLO']]

,addr_line_3,addr_line_2,StreetName_GOV,StreetName_BOLO
0,1738 W CHELTENHAM AVE,PO BOX 14299,CHELTENHAM,CHELTENHAM
1,None,PO BOX 8697,None,None
2,5620 WILLIAM PENN HWY,PO BOX 185,WILLIAM PENN,WILLIAM PENN
3,1223 E MARSHALL ST,PO BOX 980677,MARSHALL,MARSHALL
4,3200 PROVIDENCE DR,PO BOX 196604,PROVIDENCE,PROVIDENCE
...,...,...,...,...
11522,UNIV OF ALABAMA HOSPITAL,4715 WHITESBURG DR SE,WHITESBURG,WHITESBURG
11523,STE 1825,3308 CLOUDS CREEK CT,CLOUDS CREEK,CLOUDS CREEK
11524,DOCTORS OFFICE CENTER SUITE 4500,90 BERGEN ST,BERGEN,BERGEN
11525,None,8202 EXCELSIOR DR,EXCELSIOR,EXCELSIOR


In [95]:
just_a_test.columns

Index(['entity_id', 'me#', 'comm_id', 'usage', 'load_type_ind', 'addr_type',
       'addr_line_1', 'addr_line_3', 'addr_line_2', 'addr_city',
       ...
       'StreetNamePreModifier_GOV', 'USPSBoxGroupType_GOV',
       'USPSBoxGroupID_GOV', 'IntersectionSeparator_GOV',
       'SecondStreetName_GOV', 'SecondStreetNamePostType_GOV',
       'AddressNumberPrefix_GOV', 'ZipCode_GOV',
       'SecondStreetNamePostDirectional_GOV', 'SecondStreetNamePreType_GOV'],
      dtype='object', length=125)

In [106]:
pd.merge(BOLOS, maybe, on=['ME','comm_id'], how='left')[['entity_id', 'me#', 'comm_id', 'usage', 'load_type_ind', 'addr_type',
       'addr_line_1', 'addr_line_3', 'addr_line_2', 'addr_city', 'addr_state',
       'addr_zip', 'addr_plus4', 'addr_country', 'source', 'source_dtm', 'phone_number']]

,entity_id,me#,comm_id,usage,load_type_ind,addr_type,addr_line_1,addr_line_3,addr_line_2,addr_city,addr_state,addr_zip,addr_plus4,addr_country,source,source_dtm,phone_number
0,10000103,04501810326,5711907,PO,R,OF,LEXINGTON CO HLTH SVS DIST INC,RADIOLOGY DEPARTMENT,627 SPRING LAKE RD,COLUMBIA,SC,29206,NaN,NaN,ASM,2021-08-25 13:26:27.136627,NaN
1,10000233,04501810423,2065529,PO,R,OF,NaN,NaN,2 MEMORIAL MEDICAL DR,GREENVILLE,SC,29605,NaN,NaN,ASM,2021-08-25 13:26:27.136627,NaN
2,10000324,04501810491,6225203,PO,R,OF,NaN,SOMERSET HLTH & WELLNESS C,2525 COURT DR,GASTONIA,NC,28054,NaN,NaN,ASM,2021-08-25 13:26:27.136627,NaN
3,10000363,04501810521,11466799,PO,R,OF,NaN,STE 41,1130 PERRY HWY,PITTSBURGH,PA,15237,NaN,NaN,ASM,2021-08-25 13:26:27.136627,NaN
4,10000415,04501810571,9098975,PO,R,OF,NaN,STE A,130 LAURENT WAY,IRMO,SC,29063,NaN,NaN,ASM,2021-08-25 13:26:27.136627,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101602,9999284,04501801475,6584398,PO,R,OF,NaN,GREENWOOD UROLOGICAL,126 COUNTRY CLUB DR,GREENWOOD,SC,29646,NaN,NaN,ASM,2021-08-25 13:26:27.136627,NaN
101603,9999609,04501801769,6348109,PO,R,OF,"24ON PHYSICIANS, PC",THE REGIONAL MEDICAL CENTER,940 HOLLY ST,ORANGEBURG,SC,29115,NaN,NaN,ASM,2021-08-25 13:26:27.136627,NaN
101604,9999713,04501801882,6452871,PO,R,OF,NaN,NaN,861 DENALI DR,CONWAY,SC,29526,NaN,NaN,ASM,2021-08-25 13:26:27.136627,NaN
101605,9999804,04501810075,15817052,PO,R,OF,NaN,G. STEVEN AVIDON MD PA,PO BOX 621749,ORLANDO,FL,32862,NaN,NaN,ASM,2021-08-25 13:26:27.136627,NaN


In [107]:
gov

,NPI,Ind_PAC_ID,Ind_enrl_ID,lst_nm,frst_nm,mid_nm,suff,gndr,Cred,Med_sch,...,hosp_afl_3,hosp_afl_lbn_3,hosp_afl_4,hosp_afl_lbn_4,hosp_afl_5,hosp_afl_lbn_5,ind_assgn,grp_assgn,adrs_id,ME
0,1225027709,5991772014,I20040916000590,FOX,MICHAEL,D,None,M,MD,NEW YORK MEDICAL COLLEGE,...,None,None,None,None,None,None,Y,M,CA949041901GR501XXCASI300,03509741491
1,1225027709,5991772014,I20040916000590,FOX,MICHAEL,D,None,M,MD,NEW YORK MEDICAL COLLEGE,...,None,None,None,None,None,None,Y,M,CA949034879SA1550XPKWY300,03509741491
2,1225029739,345343281,I20101122001175,WASFIE,TARIK,J,None,M,None,OTHER,...,None,None,None,None,None,None,Y,M,MI485323410FL1127XCTXX401,52801700195
3,1225119407,7113925215,I20090727000165,SAIYED,ASHFAQ,None,None,M,None,OTHER,...,110073,DORMINY MEDICAL CENTER,110095,TIFT REGIONAL MEDICAL CENTER,None,None,Y,M,GA317743606OC357XXRDXX300,49589760122
4,1225119407,7113925215,I20090727000165,SAIYED,ASHFAQ,None,None,M,None,OTHER,...,110073,DORMINY MEDICAL CENTER,110095,TIFT REGIONAL MEDICAL CENTER,None,None,Y,M,GA317741804OC202XXSTXX400,49589760122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1322142,1790176931,5092035139,I20150513001683,HALVERSON,EMILY,CATHERINE,None,F,None,OTHER,...,None,None,None,None,None,None,Y,Y,MN551105189VA3200XRDXX303,02604450571
1322143,1245436591,941451975,I20121121000218,GOPAL,ARUN,K,None,M,None,UNIVERSITY OF NORTH CAROLINA AT CHAPEL HILL SC...,...,None,None,None,None,None,None,Y,Y,CA913602255TH1982XYUCC300,03601050519
1322144,1750641874,8022251925,I20130824000256,DUGAN,JASON,A,None,M,None,"UNIVERSITY OF CALIFORNIA, IRVINE, CALIFORNIA C...",...,None,None,None,None,None,None,Y,Y,CA913602255TH1982XYUCC300,00515090401
1322145,1538369830,5395046122,I20200408003958,KAMANU ELIAS,NNEMDI,None,None,F,None,YALE UNIVERSITY SCHOOL OF MEDICINE,...,None,None,None,None,None,None,Y,Y,VA224074800FR4024XRDXX300,00801990096


In [108]:
phoney

,ME,adrs_id,phn_numbr,NPI
0,00511830303,CA906404314MO101XXBLVD403,3.237281e+09,1871519561
1,00511830303,CA918013705AL220XXSTXX400,6.262819e+09,1871519561
2,00506930502,CA926634162NE1XXXXDRXX300,9.498911e+09,1790868909
3,04705920406,SC296075253GR104XXDRXX303,8.646036e+09,1508823618
4,04705920406,SC296211580AN2000XSTXX400,8.645123e+09,1508823618
...,...,...,...,...
1875059,42201137106,CA924044836SA2101XAVEX400,9.098839e+09,1417324633
1875060,70402110081,CA919116617CH751XXCTXX400,6.195026e+09,1093198392
1875061,70402110081,CA921154637SA4940XBLVD400,6.195434e+09,1093198392
1875062,70402110081,CA921181943CO250XXPLXX300,6.195224e+09,1093198392


In [112]:
gov_all = pd.merge(gov, phoney, on=['ME','adrs_id','NPI'], how='left')

In [ ]:
for row in gov_all.itertuples():
    if row.phn_numbr_x != row.phn_numbr_y